## Trees: Ensemble Methods - Boosting

Boosting is another ensemble technique to create a collection of predictors. In this technique, learners are learned sequentially with early learners fitting simple models to the data and then analyzing data for errors. In other words, we fit consecutive trees (random sample) and at every step, the goal is to solve for net error from the prior tree.

When an input is misclassified by a hypothesis, its weight is increased so that next hypothesis is more likely to classify it correctly. By combining the whole set at the end converts weak learners into better performing model.

An ensemble of trees are built one by one and individual trees are summed sequentially. The Next tree tries to recover the loss (difference between actual and predicted values) from the previous tree.

#### Gradient Boosting = Gradient Descent + Boosting.


Exercise: Load the intercampusai2019 dataset, train a random forest, extra trees model on the dataset and compare results using both random forests and gradient boosting under the *Gini and Entropy* criterions. 

<strong>Note: Also make sure to do some data cleaning, upsampling/downsampling, parameter tuning.</strong>